In [1]:
#スクレイピング
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import re

import os
import warnings
import time
import requests

from google.cloud import storage

In [2]:
race_id_text = 202309011011

In [3]:
#date = 2023年3月16日
date_text = "2023年2月12日"

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,3)

URL = "https://race.netkeiba.com/race/shutuba.html?race_id=202309011011&rf=race_list"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

C:\Users\kawam\AppData\Local\Temp\ipykernel_8744\3764771592.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [5]:
race_data_columns=[
    'race_id',
    'race_round',
    'race_title',
    'race_course',
    'weather',
    'ground_status',
    'time',
    'date',
    'where_racecourse',
    'total_horse_number',
    'frame_number_first',
    'horse_number_first',
    'frame_number_second',
    'horse_number_second',
    'frame_number_third',
    'horse_number_third',
    'tansyo',
    'hukusyo_first',
    'hukusyo_second',
    'hukusyo_third',
    'wakuren',
    'umaren',
    'wide_1_2',
    'wide_1_3',
    'wide_2_3',
    'umatan',
    'renhuku3',
    'rentan3'
    ]

horse_data_columns=[
    'race_id',
    'rank',
    'frame_number',
    'horse_number',
    'horse_id',
    'sex_and_age',
    'burden_weight',
    'rider_id',
    'goal_time',
    'goal_time_dif',
    'time_value',
    'half_way_rank',
    'last_time',
    'odds',
    'popular',
    'horse_weight',
    'tame_time',
    'tamer_id',
    'owner_id'
]

In [6]:
race_round = driver.find_elements(By.CLASS_NAME,'RaceNum')
race_round_text = [rr.text for rr in race_round]
race_round_text = race_round_text[0]
race_title = driver.find_elements(By.CLASS_NAME,'RaceName')
race_title_text = [rr.text for rr in race_title]
race_title_text = race_title_text[0]

In [7]:
race_detail1 = driver.find_elements(By.CLASS_NAME,'RaceData01')
race_details1_text = [rr.text for rr in race_detail1]
#list
race_details1_list = race_details1_text[0].split(' / ')
#list0
time_text = race_details1_list[0].strip("発走")
time_text = '発走 : '+time_text
#list1
race_details1_list1_list = race_details1_list[1].split(' ')
#listlist1
race_details1_list1_list_text = str(race_details1_list1_list[0])
text = race_details1_list1_list_text
match = re.search(r'[^\d]*(\D)', text)
if match:
    ground_type_text = match.group(1)
else:
    ground_type_text = ''

text = race_details1_list1_list_text
match = re.search(r'\d+', text)
if match:
    distance = match.group()
else:
    distance = ''
distance_int = int(distance)

text = race_details1_list1_list_text
is_obstacle = 1 if text.startswith('障') else 0
is_obstacle_int = int(is_obstacle)

is_left_right_straight_text = race_details1_list1_list[1].strip("()")
#list2
weather_text = race_details1_list[2]
#list3
ground_status_text = race_details1_list[3]

In [8]:
race_details2 = driver.find_elements(By.CLASS_NAME,'RaceData02')
race_details2_text = [rr.text for rr in race_details2]
race_details2_text = re.sub(' +', ' ', race_details2_text[0])
race_details2_list = race_details2_text.split(' ')
race_details2_list8_str = str(race_details2_list[8])
match = re.match(r'^\d+', race_details2_list8_str)
if match:
    race_details2_list[8] = match.group()
total_horse_number_text = race_details2_list[8]
where_racecourse_text = race_details2_list[0]+race_details2_list[1]+race_details2_list[2]

In [9]:
race_data = []
race_data.append({
        'race_id':race_id_text,
        'race_round': race_round_text,
        'race_title':race_title_text,
        'weather':weather_text,
        'ground_status':ground_status_text,
        'time':time_text,
        'where_racecourse':where_racecourse_text,
        'total_horse_number':total_horse_number_text,
        'ground_type':ground_type_text,
        'is_left_right_straight':is_left_right_straight_text,
        'distance':distance_int,
        'is_obstacle':is_obstacle_int,     
    })

# 辞書のリストをDataFrameに変換する
race_df = pd.DataFrame(race_data)

In [10]:
race_df['date'] = date_text

In [11]:
race_df

,race_id,race_round,race_title,weather,ground_status,time,where_racecourse,total_horse_number,ground_type,is_left_right_straight,distance,is_obstacle,date
0,202309011011,11R,フィリーズR,天候:晴,馬場:良,発走 : 15:35,1回阪神10日目,18,芝,右,1400,0,2023年2月12日


##### cleaning

In [12]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   race_id                 1 non-null      int64 
 1   race_round              1 non-null      object
 2   race_title              1 non-null      object
 3   weather                 1 non-null      object
 4   ground_status           1 non-null      object
 5   time                    1 non-null      object
 6   where_racecourse        1 non-null      object
 7   total_horse_number      1 non-null      object
 8   ground_type             1 non-null      object
 9   is_left_right_straight  1 non-null      object
 10  distance                1 non-null      int64 
 11  is_obstacle             1 non-null      int64 
 12  date                    1 non-null      object
dtypes: int64(3), object(10)
memory usage: 232.0+ bytes


In [13]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [14]:
race_df['race_round'] = race_df['race_round'].astype(int)

In [15]:
race_df["where_racecourse"] = race_df["where_racecourse"].str.replace('\d*回(..)\d*日目', r'\1')

C:\Users\kawam\AppData\Local\Temp\ipykernel_8744\3042939588.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  race_df["where_racecourse"] = race_df["where_racecourse"].str.replace('\d*回(..)\d*日目', r'\1')


In [16]:
race_df['weather'] = race_df['weather'].str.strip('天候 :')

In [17]:
weather_rain = race_df["weather"].str.extract('(小雨|雨)', expand=True)
weather_snow = race_df["weather"].str.extract('(小雪|雪)', expand=True)
weather_rain.columns ={"weather_rain"}
weather_snow.columns ={"weather_snow"}
race_df = pd.concat([race_df, weather_rain], axis=1)
race_df = pd.concat([race_df, weather_snow], axis=1)

race_df.fillna(value={'weather_rain': 0}, inplace=True)
race_df['weather_rain'] = race_df['weather_rain'].replace('小雨', 1)
race_df['weather_rain'] = race_df['weather_rain'].replace('雨', 2)
race_df.fillna(value={'weather_snow': 0}, inplace=True)
race_df['weather_snow'] = race_df['weather_snow'].replace('小雪', 1)
race_df['weather_snow'] = race_df['weather_snow'].replace('雪', 2)

In [18]:
race_df['ground_status'] = race_df['ground_status'].replace('.*(稍重).*', 4,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(重).*', 3,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(不良).*', 2,regex=True)
race_df['ground_status'] = race_df['ground_status'].replace('.*(良).*', 1,regex=True)

In [19]:
warnings.filterwarnings('ignore', 'The default value of regex')

race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')
race_df["datetime"] = race_df["date"] + race_df["time"]
race_df["datetime"] = pd.to_datetime(race_df['datetime'], format='%Y年%m月%d日%H時%M分')

In [20]:
race_df.drop(['time'], axis=1, inplace=True)
race_df.drop(['date'], axis=1, inplace=True)

In [21]:
race_df

,race_id,race_round,race_title,weather,ground_status,where_racecourse,total_horse_number,ground_type,is_left_right_straight,distance,is_obstacle,weather_rain,weather_snow,datetime
0,202309011011,11,フィリーズR,晴,1,阪神,18,芝,右,1400,0,0,0,2023-02-12 15:35:00


In [22]:
race_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   race_id                 1 non-null      int64         
 1   race_round              1 non-null      int32         
 2   race_title              1 non-null      object        
 3   weather                 1 non-null      object        
 4   ground_status           1 non-null      int64         
 5   where_racecourse        1 non-null      object        
 6   total_horse_number      1 non-null      object        
 7   ground_type             1 non-null      object        
 8   is_left_right_straight  1 non-null      object        
 9   distance                1 non-null      int64         
 10  is_obstacle             1 non-null      int64         
 11  weather_rain            1 non-null      int64         
 12  weather_snow            1 non-null      int64         

In [23]:
race_df['total_horse_number'] = race_df['total_horse_number'].astype(int)

In [24]:
race_df = race_df.drop(['race_title'],axis=1)

In [25]:
race_df.to_csv("../data/main/target_race_data.csv", index=False)

In [26]:
key_name ="../key/storage_key.json"
bucket_name = "keiba_csv_upload_bucket"
file_upload = 'target_race_data.csv'
file_name = '../data/main/target_race_data.csv'

In [27]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)

In [28]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_upload)
blob.upload_from_filename(file_name)

# horse

In [29]:
load_url = URL
html = requests.get(load_url)
soup = BeautifulSoup(html.content, "html.parser")

In [30]:
horse_list = []
horse_tables = soup.findAll("table", class_="RaceTable01")
horse_table = horse_tables[0].findAll('tr', class_="HorseList")

In [31]:
result_row  = []
horse_list_waku = []
horse_list_num = []
horse_list_horse_id = []
horse_list_agesex = []
horse_list_weight = []
horse_list_rider_id = []
horse_list_stable_id = []
horse_list_horseweight = []
horse_list_odds = []
horse_list_popluar = []
for i in range(len(horse_table)):
    result_row = horse_table[i].findAll("td")
    horse_list_waku.append(result_row[0].get_text())#枠
    horse_list_num.append(result_row[1].get_text())#番号
    horse_list_horse_id.append(result_row[3].find('a').get('href').split("/")[-1])#馬名
    horse_list_agesex.append(result_row[4].get_text())#性別年齢
    horse_list_weight.append(result_row[5].get_text())#斤量
    horse_list_rider_id.append(result_row[6].find('a').get('href').split("/")[-2])#騎手
    horse_list_stable_id.append(result_row[7].find('a').get('href').split("/")[-2])#厩舎
    horse_list_horseweight.append(result_row[8].get_text())#馬体重
    horse_list_odds.append(result_row[9].get_text())#オッズ
    horse_list_popluar.append(result_row[10].get_text())#人気

In [32]:
horse_data = []
for i in range(len(horse_table)):
    horse_data.append({
            'frame_number':horse_list_waku[i],
            'horse_number':horse_list_num[i],
            'horse_id':horse_list_horse_id[i],
            'sex_and_age':horse_list_agesex[i],
            'burden_weight':horse_list_weight[i],
            'rider_id':horse_list_rider_id[i],
            'tamer_id':horse_list_stable_id[i],
            'horse_weight':horse_list_horseweight[i],
            'odds':horse_list_odds[i],
            'popular':horse_list_popluar[i]
        })
# 辞書のリストをDataFrameに変換する
horse_df = pd.DataFrame(horse_data)
horse_df['race_id'] = race_id_text

In [33]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   frame_number   18 non-null     object
 1   horse_number   18 non-null     object
 2   horse_id       18 non-null     object
 3   sex_and_age    18 non-null     object
 4   burden_weight  18 non-null     object
 5   rider_id       18 non-null     object
 6   tamer_id       18 non-null     object
 7   horse_weight   18 non-null     object
 8   odds           18 non-null     object
 9   popular        18 non-null     object
 10  race_id        18 non-null     int64 
dtypes: int64(1), object(10)
memory usage: 1.7+ KB


##### Cleaning

In [34]:
is_senba = horse_df["sex_and_age"].str.extract('(セ)', expand=True)
is_senba.columns ={"is_senba"}
horse_df = pd.concat([horse_df, is_senba], axis=1)

is_mesu = horse_df["sex_and_age"].str.extract('(牝)', expand=True)
is_mesu.columns ={"is_mesu"}
horse_df = pd.concat([horse_df, is_mesu], axis=1)

is_osu = horse_df["sex_and_age"].str.extract('(牡)', expand=True)
is_osu.columns ={"is_osu"}
horse_df = pd.concat([horse_df, is_osu], axis=1)

In [35]:
horse_df.fillna(value={'is_osu': 0}, inplace=True)
horse_df['is_osu'] = horse_df['is_osu'].replace('牡', 1)
horse_df.fillna(value={'is_mesu': 0}, inplace=True)
horse_df['is_mesu'] = horse_df['is_mesu'].replace('牝', 1)
horse_df.fillna(value={'is_senba': 0}, inplace=True)
horse_df['is_senba'] = horse_df['is_senba'].replace('セ', 1)
## 余分な文字を削除
horse_df['sex_and_age'] = horse_df['sex_and_age'].str.strip("牝牡セ")
horse_df['sex_and_age'] = horse_df['sex_and_age'].astype(int)

In [36]:
horse_weight_dif = horse_df["horse_weight"].str.extract('\(([-|+]?\d*)\)', expand=True)
horse_weight_dif.columns ={"horse_weight_dif"}

horse_df = pd.concat([horse_df, horse_weight_dif], axis=1)

horse_df['horse_weight'] = horse_df['horse_weight'].replace('\(([-|+]?\d*)\)', '', regex=True)
horse_df['horse_weight'] = horse_df['horse_weight'].replace('計不', np.nan)
horse_df['horse_weight'] = horse_df['horse_weight'].astype(float)
horse_df['horse_weight_dif'] = horse_df['horse_weight_dif'].astype(float)

In [37]:
horse_df['frame_number'] = horse_df['frame_number'].astype(int)
horse_df['horse_number'] = horse_df['horse_number'].astype(int)
horse_df['sex_and_age'] = horse_df['sex_and_age'].astype(int)
horse_df['burden_weight'] = horse_df['burden_weight'].astype(float)
horse_df['horse_weight'] = horse_df['horse_weight'].astype(int)
horse_df['is_senba'] = horse_df['is_senba'].astype(int)
horse_df['is_mesu'] = horse_df['is_mesu'].astype(int)
horse_df['is_osu'] = horse_df['is_osu'].astype(int)
horse_df['horse_weight_dif'] = horse_df['horse_weight_dif'].astype(int)

In [38]:
horse_df['horse_id'] = horse_df['horse_id'].astype(int)
horse_df['rider_id'] = horse_df['rider_id'].astype(int)
horse_df['tamer_id'] = horse_df['tamer_id'].astype(int)

In [39]:
horse_df = horse_df.drop(['odds','popular'],axis=1)

In [40]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   frame_number      18 non-null     int32  
 1   horse_number      18 non-null     int32  
 2   horse_id          18 non-null     int32  
 3   sex_and_age       18 non-null     int32  
 4   burden_weight     18 non-null     float64
 5   rider_id          18 non-null     int32  
 6   tamer_id          18 non-null     int32  
 7   horse_weight      18 non-null     int32  
 8   race_id           18 non-null     int64  
 9   is_senba          18 non-null     int32  
 10  is_mesu           18 non-null     int32  
 11  is_osu            18 non-null     int32  
 12  horse_weight_dif  18 non-null     int32  
dtypes: float64(1), int32(11), int64(1)
memory usage: 1.2 KB


In [41]:
horse_df

,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,tamer_id,horse_weight,race_id,is_senba,is_mesu,is_osu,horse_weight_dif
0,1,1,2020102890,3,54.0,1178,1173,410,202309011011,0,1,0,4
1,1,2,2020103530,3,54.0,1015,1079,470,202309011011,0,1,0,-10
2,2,3,2020100236,3,54.0,1193,1078,466,202309011011,0,1,0,-2
3,2,4,2020100686,3,54.0,1030,1161,432,202309011011,0,1,0,-5
4,3,5,2020104496,3,54.0,1128,1157,456,202309011011,0,1,0,0
5,3,6,2020101678,3,54.0,1077,1118,434,202309011011,0,1,0,4
6,4,7,2020103076,3,54.0,1093,1026,472,202309011011,0,1,0,-6
7,4,8,2020110137,3,54.0,1032,427,500,202309011011,0,1,0,-8
8,5,9,2020100909,3,54.0,1168,1173,446,202309011011,0,1,0,-4
9,5,10,2020101590,3,54.0,1043,1051,456,202309011011,0,1,0,-8


In [42]:
horse_df.to_csv("../data/main/target_horse_data.csv", index=False)

In [43]:
key_name ="../key/storage_key.json"
bucket_name = "keiba_csv_upload_bucket"
file_upload = 'target_horse_data.csv'
file_name = '../data/main/target_horse_data.csv'

In [44]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)

In [45]:
# 保管したサービスアカウントキーを指定します
client = storage.Client.from_service_account_json(key_name)
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(file_upload)
blob.upload_from_filename(file_name)

In [46]:
horse_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   frame_number      18 non-null     int32  
 1   horse_number      18 non-null     int32  
 2   horse_id          18 non-null     int32  
 3   sex_and_age       18 non-null     int32  
 4   burden_weight     18 non-null     float64
 5   rider_id          18 non-null     int32  
 6   tamer_id          18 non-null     int32  
 7   horse_weight      18 non-null     int32  
 8   race_id           18 non-null     int64  
 9   is_senba          18 non-null     int32  
 10  is_mesu           18 non-null     int32  
 11  is_osu            18 non-null     int32  
 12  horse_weight_dif  18 non-null     int32  
dtypes: float64(1), int32(11), int64(1)
memory usage: 1.2 KB
